<a href="https://colab.research.google.com/github/ameyaoka/ml-notebooks/blob/main/movie_genre_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Movie_genre_classification




In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.7 MB/s eta 0:00:00


In [2]:
  pip install --upgrade huggingface_hub

In [3]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.2 MB/s eta 0:00:00


In [4]:
import pandas as pd

import torch

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("datadrivenscience/movie-genre-prediction")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/54000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/datadrivenscience___parquet/datadrivenscience--movie-genre-prediction-01acd85570f2b187/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 54000
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 36000
    })
})

In [8]:
dataset = dataset.class_encode_column("genre")

Casting to class labels:   0%|          | 0/54000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/36000 [00:00<?, ? examples/s]

In [9]:
train_data=dataset['train']
train_data

Dataset({
    features: ['id', 'movie_name', 'synopsis', 'genre'],
    num_rows: 54000
})

In [10]:
test_data = dataset['test']

In [11]:
validation= train_data.train_test_split(test_size=0.1, stratify_by_column="genre")

In [12]:
validation

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 48600
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 5400
    })
})

In [13]:
from datasets import DatasetDict

In [14]:
new_dataset = DatasetDict({
    'train': validation['train'],
    'test': test_data,
    'valid': validation['test']})

In [15]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 48600
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 36000
    })
    valid: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 5400
    })
})

- Dataset consist of two sets 1. train  2. test

- movie id is not required for training .
- movie_name is our first input and synopsis is our second input
- genre is out label.


- The dataset is balanced

## tokenisation


1.     Find a list of all unique words in the dataset.

2.    Associate each word/token with a unique number. This is called vocab.

3.  Encode the dataset using the mapping present in the vocab.


- Model max length defines the maximum number of tokens that a single data sample can have, i.e. in the above case, our model DistilBert can accept text sequences of up to 512 tokens long.

- Model input names are the fields that the model will take as inputs for training and inference purposes.

In [16]:
checkpoint = "bert-base-uncased"

In [17]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

- token_type_id tells model which part of input is first sentence and which part is second sentence.

In [18]:
def tokenize_function(example):
    return tokenizer(example["movie_name"], example["synopsis"], truncation=True, padding =True ,return_tensors="pt")



In [19]:
tokenized_dataset = new_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/48600 [00:00<?, ? examples/s]

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5400 [00:00<?, ? examples/s]

- **Our tokenized dataset consist of id , movie_name , synopsis genre input_id token_type_id , attnetion mask .**
- **we need attetion_mask , inputids , labels , tokent_type_ids**


In [20]:
tokenized_dataset['train'][0]

{'id': 49391,
 'movie_name': 'The Grimoire',
 'synopsis': 'After a fireman loses his fiancee in a horrendous accident, he allows his own personal demons to take over his life...Which opens the way for a real one.',
 'genre': 5,
 'input_ids': [101,
  1996,
  11844,
  26250,
  102,
  2044,
  1037,
  2543,
  2386,
  12386,
  2010,
  19455,
  1999,
  1037,
  7570,
  14343,
  15482,
  2271,
  4926,
  1010,
  2002,
  4473,
  2010,
  2219,
  3167,
  7942,
  2000,
  2202,
  2058,
  2010,
  2166,
  1012,
  1012,
  1012,
  2029,
  7480,
  1996,
  2126,
  2005,
  1037,
  2613,
  2028,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [21]:

tokenized_dataset = tokenized_dataset.rename_column('genre',"labels")

tokenized_dataset.set_format('torch')


In [22]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 48600
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36000
    })
    valid: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5400
    })
})

In [23]:
tokenized_dataset  =tokenized_dataset.remove_columns(['id','movie_name','synopsis'])

In [24]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 48600
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36000
    })
    valid: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5400
    })
})

- The labels are string so we need to use huggingface ClassLabel

In [25]:
tokenized_dataset["train"][5]

{'labels': tensor(2),
 'input_ids': tensor([  101,  1996,  2197,  3002,   102,  8117,  2912,  2003,  1037,  9454,
         26572, 20281,  2879,  2542,  1999,  1996,  2540,  1997,  1996,  2103,
          1012,  2007,  2010,  1052,  1011, 23042,  2388,  2092,  2006,  1996,
          2126,  2000,  2183,  3294,  2125,  1996, 15168,  1010,  2093,  2111,
          4607,  2010,  2166,  1011,  2169,  2007,  1037,  4872,  1011,  2169,
          2007,  1996,  2373,  2000,  6033,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [26]:
from transformers import  DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=8, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=8, collate_fn=data_collator
)


In [28]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 93]),
 'token_type_ids': torch.Size([8, 93]),
 'attention_mask': torch.Size([8, 93])}

 - label2id needs testing

In [29]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=10)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

**To make sure that everything will go smoothly during training, we pass our batch to this model:**
- **if your model outputs 4 classes, CrossEntropyLoss expects your class labels to be in the range [0,C−1]**

In [30]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(2.3056, grad_fn=<NllLossBackward0>) torch.Size([8, 10])


- **All 🤗 Transformers models will return the loss when labels are provided, and we also get the logits (two for each input in our batch, so a tensor of size 8 x 2).**

In [31]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [32]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

18225


In [33]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/18225 [00:00<?, ?it/s]

In [ ]:
model.push_to_hub('Ammok/movie_genre_2',create_pr=1)

CommitInfo(commit_url='https://huggingface.co/Ammok/movie_genre_2/commit/381e31718714d0e93534b03aa47f878a3b8b687e', commit_message='Upload BertForSequenceClassification', commit_description='', oid='381e31718714d0e93534b03aa47f878a3b8b687e', pr_url='https://huggingface.co/Ammok/movie_genre_2/discussions/1', pr_revision='refs/pr/1', pr_num=1)

In [ ]:
tokenizer.push_to_hub('Ammok/movie_genre_2',create_pr=1)

CommitInfo(commit_url='https://huggingface.co/Ammok/movie_genre_2/commit/0e601d95f5ecb54d2a8ccfc5f802079109f8504a', commit_message='Upload tokenizer', commit_description='', oid='0e601d95f5ecb54d2a8ccfc5f802079109f8504a', pr_url='https://huggingface.co/Ammok/movie_genre_2/discussions/2', pr_revision='refs/pr/2', pr_num=2)

In [ ]:
model.save_pretrained("/home")
tokenizer.save('/home')

AttributeError: ignored

In [ ]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.1 MB/s eta 0:00:00


In [ ]:
metrics = evaluate.combine([
    evaluate.load("precision", average="weighted"),
    evaluate.load("recall", average="weighted")
])

In [ ]:
import evaluate
from datasets import load_metric
metric = load_metric('recall')

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

<ipython-input-52-907c8b8135ce>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('recall')


ValueError: ignored

**loading finetuned model using transformers library**




In [34]:
text1= "The Syndicate"
text2 = "An alcoholic, his wife, a down and out American pilot and a young German form a prospecting team to search for uranium in Africa and find themselves plagued by sinister accidents."

In [35]:
tokenT=tokenizer(text1,text2,return_tensors='pt')

In [36]:
tokenT

{'input_ids': tensor([[  101,  1996, 16229,   102,  2019, 14813,  1010,  2010,  2564,  1010,
          1037,  2091,  1998,  2041,  2137,  4405,  1998,  1037,  2402,  2446,
          2433,  1037,  9824,  2075,  2136,  2000,  3945,  2005, 14247,  1999,
          3088,  1998,  2424,  3209, 17808,  2011, 16491, 13436,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [37]:
labels = ['fantasy', 'horror', 'family', 'scifi', 'action', 'crime','adventure', 'mystery', 'romance', 'thriller']

In [38]:
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}


In [39]:
tokenizer.save_pretrained("/home")

('/home/tokenizer_config.json',
 '/home/special_tokens_map.json',
 '/home/vocab.txt',
 '/home/added_tokens.json',
 '/home/tokenizer.json')

In [40]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoConfig

tokenizer = AutoTokenizer.from_pretrained("Ammok/movie_genre_2")
config = AutoConfig.from_pretrained("Ammok/movie_genre_2", label2id=label2id, id2label=id2label)
model_ftuned = AutoModelForSequenceClassification.from_pretrained("Ammok/movie_genre_2",config=config)

In [41]:
test = tokenizer(text1,text2,return_tensors='pt')
test

{'input_ids': tensor([[  101,  1996, 16229,   102,  2019, 14813,  1010,  2010,  2564,  1010,
          1037,  2091,  1998,  2041,  2137,  4405,  1998,  1037,  2402,  2446,
          2433,  1037,  9824,  2075,  2136,  2000,  3945,  2005, 14247,  1999,
          3088,  1998,  2424,  3209, 17808,  2011, 16491, 13436,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [44]:

with torch.no_grad():
    logits = model_ftuned(**test).logits



In [45]:


predicted_class_id = logits.argmax().item()
model_ftuned.config.id2label[predicted_class_id]





'thriller'

In [57]:
test_token

{'labels': tensor(0),
 'input_ids': tensor([  101,  6623,  8404,   102,  2044,  1037,  9252,  4926,  1010, 14174,
          3310,  2067,  2006,  3011,  2074,  6227,  2781,  2062,  1010,  4283,
          2000,  1037, 17208,  7561,  2081,  1999,  1037,  9097,  2436,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0,

In [97]:
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
pipe = pipeline("sentiment-analysis", tokenizer = tokenizer ,model=model_ftuned, config=config,device=0)
test_token = new_dataset['test']

# KeyDataset (only *pt*) will simply return the item in the dict returned by the dataset item
# as we're not interested in the *target* part of the dataset. For sentence pair use KeyPairDataset
for out in pipe(KeyDataset(test_token,"synopsis"), batch_size=8):

   submission = print(out)
    # {"text": "NUMBER TEN FRESH NELLY IS WAITiING ON YOU GOOD NIGHT HUSBAND"}
    # {"text": ....}
    # ....

Streaming output truncated to the last 5000 lines.
{'label': 'action', 'score': 0.4255240261554718}
{'label': 'horror', 'score': 0.7606187462806702}
{'label': 'scifi', 'score': 0.4483203887939453}
{'label': 'scifi', 'score': 0.4797954857349396}
{'label': 'scifi', 'score': 0.8051584959030151}
{'label': 'family', 'score': 0.5465758442878723}
{'label': 'action', 'score': 0.6502134799957275}
{'label': 'romance', 'score': 0.7428296208381653}
{'label': 'adventure', 'score': 0.4347831606864929}
{'label': 'action', 'score': 0.5086629986763}
{'label': 'scifi', 'score': 0.6946886777877808}
{'label': 'fantasy', 'score': 0.49545207619667053}
{'label': 'horror', 'score': 0.5007147192955017}
{'label': 'family', 'score': 0.7628970742225647}
{'label': 'scifi', 'score': 0.9559755325317383}
{'label': 'horror', 'score': 0.3942341208457947}
{'label': 'family', 'score': 0.6769758462905884}
{'label': 'fantasy', 'score': 0.7594345808029175}
{'label': 'scifi', 'score': 0.673403799533844}
{'label': 'thriller',

AttributeError: ignored

In [100]:
submission

In [81]:
pipeline

<function transformers.pipelines.pipeline(task: str = None, model: Union[str, ForwardRef('PreTrainedModel'), ForwardRef('TFPreTrainedModel'), NoneType] = None, config: Union[str, transformers.configuration_utils.PretrainedConfig, NoneType] = None, tokenizer: Union[str, transformers.tokenization_utils.PreTrainedTokenizer, ForwardRef('PreTrainedTokenizerFast'), NoneType] = None, feature_extractor: Union[str, ForwardRef('SequenceFeatureExtractor'), NoneType] = None, image_processor: Union[str, transformers.image_processing_utils.BaseImageProcessor, NoneType] = None, framework: Optional[str] = None, revision: Optional[str] = None, use_fast: bool = True, use_auth_token: Union[bool, str, NoneType] = None, device: Union[int, str, ForwardRef('torch.device'), NoneType] = None, device_map=None, torch_dtype=None, trust_remote_code: Optional[bool] = None, model_kwargs: Dict[str, Any] = None, pipeline_class: Optional[Any] = None, **kwargs) -> transformers.pipelines.base.Pipeline>

In [68]:
test_token

Dataset({
    features: ['id', 'movie_name', 'synopsis', 'genre'],
    num_rows: 36000
})

In [67]:

out

{'label': 'fantasy', 'score': 0.6098410487174988}